# Ames Residential Sales Price Prediction

### Data Triad Team

The objective of this project is to develop a robust deep learning model on this data, that can help determine from residential information of the Ames, IA neighborhoods the sale price for a house or property.

## Data Set

The first step will be to analyze and understan the data we are working with to determine what type of changes should be made in order to make the model more efficient.

In [1]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
sns.set_theme(color_codes=True)


import warnings 
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("90_percent_sample.csv", delimiter=",")

In [4]:
pd.set_option("display.max_columns", None)
df.head()

,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,Land_Slope,Neighborhood,Condition_1,Condition_2,Bldg_Type,House_Style,Overall_Qual,Overall_Cond,Year_Built,Year_Remod_Add,Roof_Style,Roof_Matl,Exterior_1st,Exterior_2nd,Mas_Vnr_Type,Mas_Vnr_Area,Exter_Qual,Exter_Cond,Foundation,Bsmt_Qual,Bsmt_Cond,Bsmt_Exposure,BsmtFin_Type_1,BsmtFin_SF_1,BsmtFin_Type_2,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,Heating,Heating_QC,Central_Air,Electrical,First_Flr_SF,Second_Flr_SF,Low_Qual_Fin_SF,Gr_Liv_Area,Bsmt_Full_Bath,Bsmt_Half_Bath,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,Kitchen_Qual,TotRms_AbvGrd,Functional,Fireplaces,Fireplace_Qu,Garage_Type,Garage_Finish,Garage_Cars,Garage_Area,Garage_Qual,Garage_Cond,Paved_Drive,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Three_season_porch,Screen_Porch,Pool_Area,Pool_QC,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price,Longitude,Latitude
0,Two_Story_1945_and_Older,Residential_Medium_Density,0,5100,Pave,Gravel,Regular,Lvl,AllPub,Inside,Gtl,Old_Town,Norm,Norm,OneFam,Two_Story,Very_Good,Good,1925,1996,Hip,CompShg,Stucco,Wd Shng,NaN,0,Typical,Good,PConc,Typical,Typical,No,Unf,7,Unf,0,588,588,GasA,Fair,Y,SBrkr,833,833,0,1666,0,0,1,0,3,1,Good,7,Typ,1,Good,Detchd,Unf,1,228,Typical,Typical,Paved,192,63,0,0,0,0,No_Pool,Minimum_Privacy,NaN,0,6,2008,WD,Normal,161000,-93.621065,42.029038
1,Two_Story_PUD_1946_and_Newer,Residential_Medium_Density,21,1890,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,Briardale,Norm,Norm,Twnhs,Two_Story,Above_Average,Good,1972,1972,Gable,CompShg,HdBoard,HdBoard,BrkFace,380,Typical,Typical,CBlock,Typical,Typical,No,ALQ,1,Unf,0,212,494,GasA,Excellent,Y,SBrkr,494,536,0,1030,0,0,1,1,3,1,Typical,6,Typ,0,No_Fireplace,Detchd,Unf,1,264,Typical,Typical,Paved,0,0,0,0,0,0,No_Pool,No_Fence,NaN,0,7,2006,WD,Normal,116000,-93.627103,42.051798
2,Two_Story_1946_and_Newer,Residential_Low_Density,62,7162,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,OneFam,Two_Story,Good,Average,2003,2004,Hip,CompShg,HdBoard,Stucco,BrkFace,190,Good,Typical,PConc,Good,Typical,No,Unf,7,Unf,0,796,796,GasA,Excellent,Y,SBrkr,806,918,0,1724,0,0,2,1,3,1,Good,8,Typ,1,Good,BuiltIn,Fin,2,616,Typical,Typical,Paved,168,57,0,0,0,0,No_Pool,No_Fence,NaN,0,5,2006,WD,Normal,196500,-93.672379,42.018990
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,60,8070,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,College_Creek,Norm,Norm,OneFam,One_Story,Below_Average,Average,1994,1995,Gable,CompShg,VinylSd,VinylSd,NaN,0,Typical,Typical,PConc,Good,Typical,No,GLQ,3,Unf,0,402,990,GasA,Excellent,Y,SBrkr,990,0,0,990,1,0,1,0,3,1,Typical,5,Typ,0,No_Fireplace,No_Garage,No_Garage,0,0,No_Garage,No_Garage,Paved,0,0,0,0,0,0,No_Pool,No_Fence,NaN,0,8,2007,WD,Normal,123600,-93.692415,42.019028
4,One_Story_1945_and_Older,Residential_Medium_Density,50,7000,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,Iowa_DOT_and_Rail_Road,Norm,Norm,OneFam,One_Story,Above_Average,Very_Good,1926,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0,Typical,Good,BrkTil,Typical,Typical,No,Rec,6,GLQ,40,555,894,GasA,Typical,Y,SBrkr,919,0,0,919,1,0,1,0,2,1,Typical,5,Typ,0,No_Fireplace,Detchd,Unf,1,195,Typical,Typical,Partial_Pavement,0,0,116,0,0,0,No_Pool,Minimum_Privacy,NaN,0,7,2008,WD,Normal,126000,-93.628409,42.022607


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 81 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MS_SubClass         2637 non-null   object 
 1   MS_Zoning           2637 non-null   object 
 2   Lot_Frontage        2637 non-null   int64  
 3   Lot_Area            2637 non-null   int64  
 4   Street              2637 non-null   object 
 5   Alley               2637 non-null   object 
 6   Lot_Shape           2637 non-null   object 
 7   Land_Contour        2637 non-null   object 
 8   Utilities           2637 non-null   object 
 9   Lot_Config          2637 non-null   object 
 10  Land_Slope          2637 non-null   object 
 11  Neighborhood        2637 non-null   object 
 12  Condition_1         2637 non-null   object 
 13  Condition_2         2637 non-null   object 
 14  Bldg_Type           2637 non-null   object 
 15  House_Style         2637 non-null   object 
 16  Overal

In [6]:
df.isnull().sum()

MS_SubClass       0
MS_Zoning         0
Lot_Frontage      0
Lot_Area          0
Street            0
                 ..
Sale_Type         0
Sale_Condition    0
Sale_Price        0
Longitude         0
Latitude          0
Length: 81, dtype: int64

## Exploratory Data Analysis

In [7]:
#Change data on Overall Quality and Overall Condition column to avoid inconsistencies between numerical and categorical

df['Overall_Qual'] = df['Overall_Qual'].replace({
    'Very_Poor': 1,
    'Poor': 2,
    'Fair': 3,
    'Below_Average': 4,
    'Average': 5,
    'Above_Average': 6,
    'Good': 7,
    'Very_Good': 8,
    'Excellent': 9,
    'Very_Excellent': 10
})
df['Overall_Cond'] = df['Overall_Cond'].replace({
    'Very_Poor': 1,
    'Poor': 2,
    'Fair': 3,
    'Below_Average': 4,
    'Average': 5,
    'Above_Average': 6,
    'Good': 7,
    'Very_Good': 8,
    'Excellent': 9,
    'Very_Excellent': 10
})

In [8]:
#Categorical Data Selection 
df_categorical = df[['MS_SubClass', 'MS_Zoning', 'Street', 'Alley', 'Lot_Shape',
                    'Land_Contour', 'Utilities', 'Lot_Config', 'Land_Slope', 
                    'Neighborhood', 'Condition_1', 'Condition_2', 'Bldg_Type',
                    'House_Style', 'Roof_Style', 'Roof_Matl', 'Exterior_1st',
                    'Exterior_2nd', 'Mas_Vnr_Type', 'Exter_Qual', 'Exter_Cond',
                    'Foundation', 'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure',
                    'BsmtFin_Type_1', 'BsmtFin_Type_2', 'Heating', 'Heating_QC',
                    'Central_Air', 'Electrical', 'Kitchen_Qual', 'Functional',
                    'Fireplace_Qu', 'Garage_Type', 'Garage_Finish', 'Garage_Qual',
                    'Garage_Cond', 'Paved_Drive', 'Pool_QC', 'Fence', 'Misc_Feature',
                    'Sale_Type', 'Sale_Condition', ]]
df_categorical.head()

,MS_SubClass,MS_Zoning,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,Land_Slope,Neighborhood,Condition_1,Condition_2,Bldg_Type,House_Style,Roof_Style,Roof_Matl,Exterior_1st,Exterior_2nd,Mas_Vnr_Type,Exter_Qual,Exter_Cond,Foundation,Bsmt_Qual,Bsmt_Cond,Bsmt_Exposure,BsmtFin_Type_1,BsmtFin_Type_2,Heating,Heating_QC,Central_Air,Electrical,Kitchen_Qual,Functional,Fireplace_Qu,Garage_Type,Garage_Finish,Garage_Qual,Garage_Cond,Paved_Drive,Pool_QC,Fence,Misc_Feature,Sale_Type,Sale_Condition
0,Two_Story_1945_and_Older,Residential_Medium_Density,Pave,Gravel,Regular,Lvl,AllPub,Inside,Gtl,Old_Town,Norm,Norm,OneFam,Two_Story,Hip,CompShg,Stucco,Wd Shng,NaN,Typical,Good,PConc,Typical,Typical,No,Unf,Unf,GasA,Fair,Y,SBrkr,Good,Typ,Good,Detchd,Unf,Typical,Typical,Paved,No_Pool,Minimum_Privacy,NaN,WD,Normal
1,Two_Story_PUD_1946_and_Newer,Residential_Medium_Density,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,Briardale,Norm,Norm,Twnhs,Two_Story,Gable,CompShg,HdBoard,HdBoard,BrkFace,Typical,Typical,CBlock,Typical,Typical,No,ALQ,Unf,GasA,Excellent,Y,SBrkr,Typical,Typ,No_Fireplace,Detchd,Unf,Typical,Typical,Paved,No_Pool,No_Fence,NaN,WD,Normal
2,Two_Story_1946_and_Newer,Residential_Low_Density,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,OneFam,Two_Story,Hip,CompShg,HdBoard,Stucco,BrkFace,Good,Typical,PConc,Good,Typical,No,Unf,Unf,GasA,Excellent,Y,SBrkr,Good,Typ,Good,BuiltIn,Fin,Typical,Typical,Paved,No_Pool,No_Fence,NaN,WD,Normal
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,College_Creek,Norm,Norm,OneFam,One_Story,Gable,CompShg,VinylSd,VinylSd,NaN,Typical,Typical,PConc,Good,Typical,No,GLQ,Unf,GasA,Excellent,Y,SBrkr,Typical,Typ,No_Fireplace,No_Garage,No_Garage,No_Garage,No_Garage,Paved,No_Pool,No_Fence,NaN,WD,Normal
4,One_Story_1945_and_Older,Residential_Medium_Density,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,Iowa_DOT_and_Rail_Road,Norm,Norm,OneFam,One_Story,Gable,CompShg,VinylSd,VinylSd,NaN,Typical,Good,BrkTil,Typical,Typical,No,Rec,GLQ,GasA,Typical,Y,SBrkr,Typical,Typ,No_Fireplace,Detchd,Unf,Typical,Typical,Partial_Pavement,No_Pool,Minimum_Privacy,NaN,WD,Normal


In [9]:
#Numerical Data Selection
df_numerical = df[[
    "Lot_Frontage", "Lot_Area", 'Overall_Qual' , 'Overall_Cond', 'Year_Built',
    'Year_Remod_Add', 'Mas_Vnr_Area', 'BsmtFin_SF_1', 'BsmtFin_SF_2', 'Bsmt_Unf_SF',
    'Total_Bsmt_SF', 'First_Flr_SF', 'Second_Flr_SF', 'Low_Qual_Fin_SF',
    'Gr_Liv_Area', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath', 'Full_Bath',
    'Half_Bath', 'Bedroom_AbvGr', 'Kitchen_AbvGr', 'TotRms_AbvGrd',
    'Fireplaces', 'Garage_Cars', 'Garage_Area', 'Wood_Deck_SF',
    'Open_Porch_SF', 'Enclosed_Porch', 'Three_season_porch', 'Screen_Porch',
    'Pool_Area', 'Misc_Val', 'Sale_Price', 'Longitude', 'Latitude', 'Mo_Sold', 'Year_Sold', 
]]

df_numerical.head()

,Lot_Frontage,Lot_Area,Overall_Qual,Overall_Cond,Year_Built,Year_Remod_Add,Mas_Vnr_Area,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Low_Qual_Fin_SF,Gr_Liv_Area,Bsmt_Full_Bath,Bsmt_Half_Bath,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,TotRms_AbvGrd,Fireplaces,Garage_Cars,Garage_Area,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Three_season_porch,Screen_Porch,Pool_Area,Misc_Val,Sale_Price,Longitude,Latitude,Mo_Sold,Year_Sold
0,0,5100,8,7,1925,1996,0,7,0,588,588,833,833,0,1666,0,0,1,0,3,1,7,1,1,228,192,63,0,0,0,0,0,161000,-93.621065,42.029038,6,2008
1,21,1890,6,7,1972,1972,380,1,0,212,494,494,536,0,1030,0,0,1,1,3,1,6,0,1,264,0,0,0,0,0,0,0,116000,-93.627103,42.051798,7,2006
2,62,7162,7,5,2003,2004,190,7,0,796,796,806,918,0,1724,0,0,2,1,3,1,8,1,2,616,168,57,0,0,0,0,0,196500,-93.672379,42.018990,5,2006
3,60,8070,4,5,1994,1995,0,3,0,402,990,990,0,0,990,1,0,1,0,3,1,5,0,0,0,0,0,0,0,0,0,0,123600,-93.692415,42.019028,8,2007
4,50,7000,6,8,1926,1998,0,6,40,555,894,919,0,0,919,1,0,1,0,2,1,5,0,1,195,0,0,116,0,0,0,0,126000,-93.628409,42.022607,7,2008


In [10]:
from sklearn import preprocessing

# List of categorical columns
categorical_columns = [
    'MS_SubClass', 'MS_Zoning', 'Street', 'Alley', 'Lot_Shape', 'Land_Contour',
    'Utilities', 'Lot_Config', 'Land_Slope', 'Neighborhood', 'Condition_1', 
    'Condition_2', 'Bldg_Type', 'House_Style', 'Roof_Style', 'Roof_Matl', 
    'Exterior_1st', 'Exterior_2nd', 'Mas_Vnr_Type', 'Exter_Qual', 'Exter_Cond', 
    'Foundation', 'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1', 
    'BsmtFin_Type_2', 'Heating', 'Heating_QC', 'Central_Air', 'Electrical', 
    'Kitchen_Qual', 'Functional', 'Fireplace_Qu', 'Garage_Type', 'Garage_Finish', 
    'Garage_Qual', 'Garage_Cond', 'Paved_Drive', 'Pool_QC', 'Fence', 
    'Misc_Feature', 'Sale_Type', 'Sale_Condition'
]

# Apply LabelEncoder to each column and show before/after values
for col in categorical_columns:
    # Display original values
    print(f"Original values in '{col}':", df[col].unique())
    
    # Fit and transform the column
    label_encoder = preprocessing.LabelEncoder()
    df[col] = label_encoder.fit_transform(df[col])
    
    # Display encoded values
    print(f"Encoded values in '{col}':", df[col].unique())
    print('-' * 40)


Original values in 'MS_SubClass': ['Two_Story_1945_and_Older' 'Two_Story_PUD_1946_and_Newer'
 'Two_Story_1946_and_Newer' 'One_Story_1946_and_Newer_All_Styles'
 'One_Story_1945_and_Older' 'One_Story_PUD_1946_and_Newer'
 'Duplex_All_Styles_and_Ages' 'Two_Family_conversion_All_Styles_and_Ages'
 'Split_or_Multilevel' 'Split_Foyer'
 'One_and_Half_Story_Finished_All_Ages'
 'One_and_Half_Story_Unfinished_All_Ages' 'Two_and_Half_Story_All_Ages'
 'PUD_Multilevel_Split_Level_Foyer'
 'One_Story_with_Finished_Attic_All_Ages'
 'One_and_Half_Story_PUD_All_Ages']
Encoded values in 'MS_SubClass': [12 14 13  2  1  3  0 11 10  9  5  7 15  8  4  6]
----------------------------------------
Original values in 'MS_Zoning': ['Residential_Medium_Density' 'Residential_Low_Density'
 'Residential_High_Density' 'Floating_Village_Residential' 'C_all' 'I_all'
 'A_agr']
Encoded values in 'MS_Zoning': [6 5 4 2 1 3 0]
----------------------------------------
Original values in 'Street': ['Pave' 'Grvl']
Encoded values 

In [11]:
X = df.drop('Sale_Price', axis=1)
y = df['Sale_Price']

### Linear Regression

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=0)

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Step 3: Initialize and train the Linear Regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Step 4: Make predictions on the test set
y_pred = lr.predict(X_test)

# Step 5: Calculate performance metrics on the test set
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error on Test Set: {mae:.2f}')
print(f'Mean Squared Error on Test Set: {mse:.2f}')
print(f'R-squared on Test Set: {r2:.2f}')

Mean Absolute Error on Test Set: 19787.26
Mean Squared Error on Test Set: 1370465347.08
R-squared on Test Set: 0.80


## SVM

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Step 3: Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Initialize the SVR model with the RBF kernel
svr = SVR(kernel='rbf')

# Step 5: Define the hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.2, 0.5],
    'gamma': ['scale', 'auto']
}

# Step 6: Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Step 7: Get the best model from GridSearchCV
best_svr = grid_search.best_estimator_

# Step 8: Make predictions on the test set
y_pred = best_svr.predict(X_test_scaled)

# Step 9: Calculate performance metrics on the test set
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Hyperparameters: {grid_search.best_params_}')
print(f'Mean Absolute Error on Test Set: {mae:.2f}')
print(f'Mean Squared Error on Test Set: {mse:.2f}')
print(f'R-squared on Test Set: {r2:.2f}')


Best Hyperparameters: {'C': 100, 'epsilon': 0.01, 'gamma': 'auto'}
Mean Absolute Error on Test Set: 52113.54
Mean Squared Error on Test Set: 6361710220.17
R-squared on Test Set: 0.08


## Decision Tree

In [15]:
from sklearn.tree import DecisionTreeRegressor

# Use 'squared_error' for the criterion
regressor = DecisionTreeRegressor(criterion='squared_error', max_depth=3, random_state=0)

# Fit the model
regressor.fit(X_train, y_train)

# Make predictions
y_pred = regressor.predict(X_test)


In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Absolute Error: 31175.73
Mean Squared Error: 1907512301.00
Root Mean Squared Error: 43675.08
R-squared: 0.72


## Random Forest

In [17]:
from sklearn.ensemble import RandomForestRegressor

# Initialize a RandomForestRegressor with n_estimators = 300
regressor = RandomForestRegressor(n_estimators=300, random_state=0)

# Fit the model to the training set
regressor.fit(X_train, y_train)

# Make predictions
y_pred = regressor.predict(X_test)

In [18]:
# Most important scores of the model's features
feature_scores = pd.Series(regressor.feature_importances_, index=X_train.columns).sort_values(ascending=False)

pd.set_option('display.max_rows', None)

# Display feature scores
print(feature_scores)

Overall_Qual          0.599095
Gr_Liv_Area           0.096839
First_Flr_SF          0.039677
Total_Bsmt_SF         0.038026
Second_Flr_SF         0.019448
Garage_Area           0.018280
Lot_Area              0.015105
Kitchen_Qual          0.014560
Bsmt_Qual             0.014444
Garage_Cars           0.014218
Year_Built            0.010135
Longitude             0.008205
Year_Remod_Add        0.008196
Latitude              0.008039
Full_Bath             0.007555
Bsmt_Unf_SF           0.006569
Mas_Vnr_Area          0.006251
Fireplaces            0.005312
Open_Porch_SF         0.004067
Lot_Frontage          0.004036
Overall_Cond          0.003921
Wood_Deck_SF          0.003545
Garage_Type           0.003298
Mo_Sold               0.003183
TotRms_AbvGrd         0.003101
Neighborhood          0.002599
Screen_Porch          0.002559
Bsmt_Full_Bath        0.002471
MS_Zoning             0.002187
Exter_Qual            0.002024
Bsmt_Exposure         0.002019
Sale_Condition        0.001924
BsmtFin_

In [19]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Absolute Error: 15751.14
Mean Squared Error: 658577807.90
Root Mean Squared Error: 25662.77
R-squared: 0.90


## XG Boost

In [20]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [21]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=1000, learning_rate=0.001, random_state=0)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Absolute Error: 29013.66
Mean Squared Error: 1840591138.13
Root Mean Squared Error: 42902.11
R-squared: 0.73


In [23]:
feature_scores = pd.Series(xgb.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print(feature_scores)

Overall_Qual          0.486642
Garage_Cars           0.053597
First_Flr_SF          0.036847
Gr_Liv_Area           0.034610
Kitchen_Qual          0.031577
Land_Contour          0.030302
Central_Air           0.018713
Total_Bsmt_SF         0.018086
Bsmt_Qual             0.016190
Second_Flr_SF         0.014994
Garage_Type           0.014971
Latitude              0.013602
Fireplaces            0.013317
Exter_Qual            0.012543
Full_Bath             0.011217
Neighborhood          0.010765
Year_Remod_Add        0.009917
Lot_Area              0.008650
MS_Zoning             0.008279
BsmtFin_Type_1        0.007970
Year_Built            0.007941
BsmtFin_SF_2          0.007489
Longitude             0.007317
Mas_Vnr_Type          0.007143
Garage_Area           0.007018
Lot_Shape             0.006627
Paved_Drive           0.006447
Overall_Cond          0.005543
Foundation            0.005333
Bsmt_Full_Bath        0.005329
Bsmt_Unf_SF           0.004511
Half_Bath             0.004488
House_St

## Best algorithm

In [24]:
from sklearn.metrics import accuracy_score,log_loss

In [25]:
lr = LinearRegression()
dt = DecisionTreeRegressor(max_depth=3, random_state=0)
rfr = RandomForestRegressor(n_estimators=100, random_state=0)
svr = SVR(kernel='poly', C=1, degree=3)
xgbr = XGBRegressor(n_estimators=1000, learning_rate=0.01, random_state=0)

In [26]:
algos = [lr, rfr, svr, dt, xgbr]
ml_algo = ['Linear Regression', 'Random Forest', 'SVR', 'Decision Tree', 'XGBoost']

In [27]:
# Step 3: Loop through all the algorithms and evaluate their performance
for i, j in zip(algos, ml_algo):
    i.fit(X_train, y_train)  # Fit the model on training data
    pred = i.predict(X_test)  # Predict on test data

    # Calculate regression metrics
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, pred)

    # Print the results
    print(j, ':\n')
    print("Test metrics")
    print(f'Mean Absolute Error: {mae:.2f}')
    print(f'Mean Squared Error: {mse:.2f}')
    print(f"Root Mean Squared Error: {rmse:.2f}")
    print(f'R-squared: {r2:.2f}')
    print('=' * 40)
    
    pred = i.predict(X_train)  # Predict on test data

    # Calculate regression metrics
    mae = mean_absolute_error(y_train, pred)
    mse = mean_squared_error(y_train, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_train, pred)

    # Print the results
    print(j, ':\n')
    print("Train metrics")
    print(f'Mean Absolute Error: {mae:.2f}')
    print(f'Mean Squared Error: {mse:.2f}')
    print(f"Root Mean Squared Error: {rmse:.2f}")
    print(f'R-squared: {r2:.2f}')
    print('=' * 40)

Linear Regression :

Test metrics
Mean Absolute Error: 19787.26
Mean Squared Error: 1370465347.08
Root Mean Squared Error: 37019.80
R-squared: 0.80
Linear Regression :

Train metrics
Mean Absolute Error: 17756.28
Mean Squared Error: 740154429.88
Root Mean Squared Error: 27205.78
R-squared: 0.88
Random Forest :

Test metrics
Mean Absolute Error: 15729.70
Mean Squared Error: 647382431.26
Root Mean Squared Error: 25443.71
R-squared: 0.91
Random Forest :

Train metrics
Mean Absolute Error: 6149.49
Mean Squared Error: 107941849.95
Root Mean Squared Error: 10389.51
R-squared: 0.98
SVR :

Test metrics
Mean Absolute Error: 57264.13
Mean Squared Error: 7096139404.22
Root Mean Squared Error: 84238.59
R-squared: -0.03
SVR :

Train metrics
Mean Absolute Error: 55764.28
Mean Squared Error: 6605002656.48
Root Mean Squared Error: 81271.17
R-squared: -0.06
Decision Tree :

Test metrics
Mean Absolute Error: 31175.73
Mean Squared Error: 1907512301.00
Root Mean Squared Error: 43675.08
R-squared: 0.72
Dec

In [29]:
import pandas as pd

# Assume df is your DataFrame and 'variable_name' is the column of interest
min_value = df['Sale_Price'].min()
max_value = df['Sale_Price'].max()

print(f"Minimum value of Price Sales: {min_value}")
print(f"Maximum value of Price Sales: {max_value}")


Minimum value of Price Sales: 12789
Maximum value of Price Sales: 755000


In [34]:
for model, name in zip([best_rf, best_xgb], ['Random Forest', 'XGBoost']):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f'{name} - Best Hyperparameters: {model.get_params()}')
    print(f'Mean Absolute Error: {mae:.2f}')
    print(f'Mean Squared Error: {mse:.2f}')
    print(f'Root Mean Squared Error: {rmse:.2f}')
    print(f'R-squared: {r2:.2f}')
    print('=' * 40)


Random Forest - Best Hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 15, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}
Mean Absolute Error: 12092.23
Mean Squared Error: 466191149.56
Root Mean Squared Error: 21591.46
R-squared: 0.92
XGBoost - Best Hyperparameters: {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.8, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.1, 'max_bin': None, 'max_cat_threshold': Non